In [1]:
from qiskit import QuantumCircuit
from qiskit.test.mock import FakeMontreal
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passmanager import PassManager

from tweedledum.target import Device
from tweedledum.qiskit.passes.mapping import Mapping
from tweedledum.qiskit.passes.optimization import SteinerResynth
from tweedledum.qiskit import from_qiskit, to_qiskit

backend = FakeMontreal()
coupling_map = CouplingMap(backend.configuration().coupling_map)
device = Device.from_edge_list(backend.configuration().coupling_map)

map_ = Mapping(coupling_map)
opt_ = SteinerResynth(coupling_map)

In [2]:
with os.scandir('../qasm/') as entries:
    for entry in entries:
        if not entry.name.endswith('.qasm'):
            continue
        print(f'[{entry.name}]')
        circuit = from_qiskit(QuantumCircuit.from_qasm_file(entry.path))
        mapped = map_.run(circuit)
        mapped = opt_.run(mapped)
        mapped = to_qiskit(mapped, circuit_type='gatelist')
        print('CX overhead:', len(circuit) - mapped.num_ops(), '\n')

[xor5_254.qasm]
CX overhead: 15 

[4_49_16.qasm]
CX overhead: 354 

[sys6-v0_111.qasm]
CX overhead: 435 

[urf5_280.qasm]


KeyboardInterrupt: 